# pb文件 可以事先将模型导入到pd文件中 需要的时候导出

In [1]:
import tensorflow as tf
import numpy as np

I:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 下面是使用deep dream生成 224*224的图片

In [1]:
model_fn = 'tensorflow_inception_graph.pb'

In [ ]:
with tf.gfile.FastGFile(model_fn , 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
t_input = tf.placeholder(np.float32 , name='input') #导入的时候需要给网络指定一个输入图像

imagenet_mean = 117.0 #inception模型已经减去均值

t_preprocessed = tf.expand_dims(t_input - imagenet_mean , 0) #图片RGB减去均值

tf.import_graph_def(graph_def , {'input':t_preprocessed}) #导入模型

In [ ]:
layers = [op.name for op in graph_def.get_operations() if op.type == 'Conv2D' and 'import/' in op.name]

print(len(layers)) #多少层

#后面将最大化上面的卷积层某一个通道的平均值 以达到生成图像的目的
name = 'mixed4d_3x3_bottleneck_pre_relu'

#输出一个卷积层的形状
graph_def.get_tensor_by_name('import/'+name+':0').get_shape()
#>>>(? ? ? 144)

In [ ]:
name = 'mixed4d_3x3_bottleneck_pre_relu'
channel = 139 #任意选择一个通道进行最大化 最大化通道激活值 更新的不是网络权重 而是图像像素值

layer_output = graph_df.get_tensor_by_name('input/%s:0' % name)

#原始的图像噪声
img_noise = np.random.uniform(size=(224,224,3)) + 100.0 #像素为100加噪声

#渲染函数 img_noise表示图像的优化起点
render_naive(layer_output[:,:,:,channel] , img_noise , iter_n=20)

In [ ]:
def render_naive(t_obj , img0 , iter_n=20 , step=1.0):
    '''
    渲染函数
    '''
    #优化目标
    t_score = tf.reduce_mean(t_obj)
    
    #计算优化目标对输入图像的梯度
    t_grad = tf.gradients(t_score , t_input)[0]
    
    #创建新图
    img = img0.copy()
    
    for i in range(iter_n):
        g , score = sess.run([t_grad , t_score] , feed_dict={t_input : img})
        
        #简单对梯度进行正则化处理
        g /= g.std() + 1e-8
        
        img += g*step #应用到图像上
        
        print(score) #会观察到score逐渐变大

# 向更大尺寸图片进发  
## 上面生成的图片是224*224的 还可以生成更大的  
## 但是存在显存被占用过多的问题  
## 故每次不对整张图片优化 而是将图片分成几个部分  
## 每次只对其中的一个部分做优化

In [ ]:
def calc_grad_tiled(img , t_grad , tile_size=512):
    #每次只对tile_size*tile_size大小图像计算梯度
    sz = tile_size
    
    h , w = img.shape[:2]
    
    sx , sy = np.random.randint(sz , size=2)
    
    #防止在tile的边缘产生边缘效应
    img_shift = np.roll(img , sx , axis=1) #先在水平方向滚动 滚动距离随机
    img_shift = np.roll(img , sy , axis=0) #后在垂直方向滚动 滚动距离随机
    
    grad = np.zeros_like(img)
    
    for y in range(0 , max(h-sz//2 , sz) , sz):
        for x in range(0 , max(w - sz//2 , sz) , sz):
            pass

In [2]:
np.random.randint(512)

213

In [4]:
np.roll([1,2],2)

array([1, 2])